# Coourisation of black and white photos - Notebook
### Haaris Hayat

This notebook can be used to generate all of the results included in the associated PDF report for converting black and white photos to colour.
All the results were obtained on a Macbook with the following specifications:

  Processor Name:	8-Core Intel Core i9
  Processor Speed:	2.4 GHz
  Number of Processors:	1
  Total Number of Cores:	8
  L2 Cache (per Core):	256 KB
  L3 Cache:	16 MB
  Hyper-Threading Technology:	Enabled
  Memory:	32 GB

On this system, the entire script ran in around 12 minutes using default resolutions.

Note that several functions contain the option to use a GPU, I have left these in case this helps to speed up the process.
However since I do not have access to a dedicated GPU, I have not tested this functionality.

## Initial setup
These steps only need to be performed once. Firstly,

In [1]:
from datetime import datetime
a=datetime.now()
# The colorization model by Richard Zhang is not available as a package
# We therefore need to clone the repo into our project
!git clone https://github.com/richzhang/colorization.git

fatal: destination path 'colorization' already exists and is not an empty directory.


In [2]:
# Install all the required packages including DeOldify
!pip install -r requirements.txt


  Using cached torch-1.13.1-cp39-none-macosx_10_9_x86_64.whl (135.3 MB)
  Using cached scikit_image-0.19.3-cp39-cp39-macosx_10_13_x86_64.whl (13.3 MB)
  Using cached numpy-1.24.1-cp39-cp39-macosx_10_9_x86_64.whl (19.8 MB)
  Using cached matplotlib-3.6.2-cp39-cp39-macosx_10_12_x86_64.whl (7.3 MB)
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Using cached fastai-1.0.51-py3-none-any.whl (214 kB)
  Using cached tensorboardX-2.5.1-py2.py3-none-any.whl (125 kB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached yt_dlp-2023.1.2-py2.py3-none-any.whl (2.8 MB)
  Using cached opencv_python-4.7.0.68-cp37-abi3-macosx_10_13_x86_64.whl (51.7 MB)
  Using cached Pillow-9.4.0-cp39-cp39-macosx_10_10_x86_64.whl (3.3 MB)
  Using cached imgaug-0.4.0-py2.py3-none-any.whl (948 kB)
  Using cached pandas-1.5.2-cp39-cp39-macosx_10_9_x86_64.whl (12.1 MB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached fastprogress-1.0.3-py3-none-any.whl (12 kB)
  Using

## Image preparation
The first step is to prepare our test images. These are a set of personal images in colour. We therefore need to convert them to black and white images.

In [3]:
# Import necessary packages
from os import listdir, mkdir
from os.path import join, basename
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

The image data is stored in a folder called 'test_images' in the same directory as this notebook.

The results of this step are store in a folder called 'test_images_bw'

In [ ]:
# Get path to all images
images = [join('test_images/', f) for f in listdir('test_images/') if basename(f).split('.')[1] in ['jpeg', 'jpg', 'png']]

# Create folder to store results if one does not exist
try:
	mkdir('test_images_bw')
except FileExistsError:
	pass

# Convert to greyscale
for file in images:
	image_name = basename(file)
	new_name = image_name.split('.')[0]+'_bw.'+image_name.split('.')[1]
	img = Image.open(file)
	img = img.convert('L')
	img.save(join('test_images_bw/',new_name))

## Denoising old images
Some of the old photos that have been digitised need to be cleaned.
The following blocks of code detail how these images have been processed.

These photos are stored in a folder called 'old_images' in the same directory as this notebook.

In [ ]:
# Import tools used for cleaning
from skimage import io
from skimage.filters.rank import mean, median, minimum, maximum
from skimage.exposure import adjust_gamma, adjust_log, adjust_sigmoid

### Image 1
The first image is that of my grandfather.  This image looks desaturated.
The results from the code blocks below are included in Figure 5 of the pdf.

In [ ]:
# Original image
img = io.imread('old_images/IMG-20221229-WA0014.jpg', as_gray=True)
io.imshow(img)

In [ ]:
# Try max filter
max = maximum(img, selem=np.ones((3,3)))
io.imshow(max)

In [ ]:
# Try min filter
min = minimum(img, selem=np.ones((7,7)))
io.imshow(min)

In [ ]:
# Try mean filter
me = mean(img, selem=np.ones((5,5)))
io.imshow(me)

In [ ]:
# Try median filter
med = median(img, selem=np.ones((5,5)))
io.imshow(med)

In [ ]:
# Try gamma correction
gm = adjust_gamma(img,2)
io.imshow(gm)

In [ ]:
# Try log adjustment
lg = adjust_log(img,1)
io.imshow(lg)

The best results were from the gamma correction, so we will save this version alongside the original.

In [ ]:
gm = (gm*255).astype('uint8')
io.imsave('old_images/IMG-20221229-WA0014_gamma.jpg', gm)

### Image 2
The second image is that of my grandparents. This was a scan of a canvas print and thus contains white spekcs which we will try to remove.
The results from the code blocks below are included in Figure 4 of the pdf.


In [ ]:
# Original
img2 = io.imread('old_images/IMG-20230101-WA0018.jpg', as_gray=True)
io.imshow(img2)

In [ ]:
# Try mean filter
mean2 = mean(img2, selem=np.ones((3,3)))
io.imshow(mean2)

In [ ]:
# Try median filter
median2 = median(img2, selem=np.ones((3,3)))
io.imshow(median2)

In [ ]:
# Try min filter
min2 = minimum(img2, selem=np.ones((3,3)))
io.imshow(min2)

In [ ]:
# Minimum and mean filter had best results
# We will save these alongside the original

mean2=mean2.astype('uint8')
min2=min2.astype('uint8')
io.imsave('old_images/IMG-20230101-WA0018_mean.jpg', mean2)
io.imsave('old_images/IMG-20230101-WA0018_min.jpg', min2)

## Colorization model
We will start by applying the colorization repository by Zhang et al. to our images.

In [ ]:
from colorization.colorizers import *

In [ ]:
# Initialise colourisers
use_gpu = False
# load colorizers
colorizer_eccv16 = eccv16(pretrained=True).eval()
colorizer_siggraph17 = siggraph17(pretrained=True).eval()
if use_gpu:
	colorizer_eccv16.cuda()
	colorizer_siggraph17.cuda()

In [ ]:
# Set up folder to store results
try:
	mkdir('out_colorizer/')
except FileExistsError:
	pass

In [ ]:
# All the synthetic greyscale images:
images = [join('test_images_bw/', f) for f in listdir('test_images_bw/') if basename(f).split('.')[1] in ['jpeg', 'jpg', 'png']]

# Cycle through black and white images
for file in images:
	img = load_img(file)
	#load original colour for comparison
	org = load_img(join('test_images/', basename(file).replace('_bw','')))

	# default size to process images is 256x256
	# grab L channel in both original ("orig") and resized ("rs") resolutions
	# Note I tried a number of resolutions but 256x256 seemed to give the best results
	(tens_l_orig, tens_l_rs) = preprocess_img(img, HW=(256,256))
	if use_gpu:
		tens_l_rs = tens_l_rs.cuda()

	# colorizer outputs 256x256 ab map
	# resize and concatenate to original L channel
	img_bw = postprocess_tens(tens_l_orig, torch.cat((0*tens_l_orig,0*tens_l_orig),dim=1))
	out_img_eccv16 = postprocess_tens(tens_l_orig, colorizer_eccv16(tens_l_rs).cpu())
	out_img_siggraph17 = postprocess_tens(tens_l_orig, colorizer_siggraph17(tens_l_rs).cpu())

	# Names of output files
	save_name_eccv16 = 'out_colorizer/eccv16_' + basename(file)
	save_name_sig17 = 'out_colorizer/sig17_' + basename(file)

	# Save outputs which are used in Figure 6 of the pdf
	plt.imsave(save_name_eccv16, out_img_eccv16)
	plt.imsave(save_name_sig17, out_img_siggraph17)

	# Plot results to allow comparison
	plt.figure(figsize=(12,8))
	plt.subplot(2,2,1)
	plt.imshow(org)
	plt.title('Original')
	plt.axis('off')

	plt.subplot(2,2,2)
	plt.imshow(img_bw)
	plt.title('Input')
	plt.axis('off')

	plt.subplot(2,2,3)
	plt.imshow(out_img_eccv16)
	plt.title('Output (ECCV 16)')
	plt.axis('off')

	plt.subplot(2,2,4)
	plt.imshow(out_img_siggraph17)
	plt.title('Output (SIGGRAPH 17)')
	plt.axis('off')

	plt.show()


The results of the code block above form the second and third columns of Figure 6 in the pdf.

We will now move on to applying the colorizer to the old photos.
This was performed for completeness, however to keep the pdf report concise, we have only focussed on the use of the DeOldify model on old photos.

In [ ]:
images_old = [join('old_images/', f) for f in listdir('old_images/') if basename(f).split('.')[1] in ['jpeg', 'jpg', 'png']]

# Cycle through old black and white images as before
for file in images_old:
	img = load_img(file)

	# default size to process images is 256x256
	# grab L channel in both original ("orig") and resized ("rs") resolutions
	(tens_l_orig, tens_l_rs) = preprocess_img(img, HW=(256,256))
	if use_gpu:
		tens_l_rs = tens_l_rs.cuda()

	# colorizer outputs 256x256 ab map
	# resize and concatenate to original L channel
	img_bw = postprocess_tens(tens_l_orig, torch.cat((0*tens_l_orig,0*tens_l_orig),dim=1))
	out_img_eccv16 = postprocess_tens(tens_l_orig, colorizer_eccv16(tens_l_rs).cpu())
	out_img_siggraph17 = postprocess_tens(tens_l_orig, colorizer_siggraph17(tens_l_rs).cpu())

	save_name_eccv16 = 'out_colorizer/eccv16_' + basename(file)
	save_name_sig17 = 'out_colorizer/sig17_' + basename(file)

	plt.imsave(save_name_eccv16, out_img_eccv16)
	plt.imsave(save_name_sig17, out_img_siggraph17)

	# Plot results to allow comparison
	plt.figure(figsize=(12,8))
	plt.subplot(2,2,1)
	plt.imshow(img)
	plt.title('Original')
	plt.axis('off')

	plt.subplot(2,2,2)
	plt.imshow(img_bw)
	plt.title('Input')
	plt.axis('off')

	plt.subplot(2,2,3)
	plt.imshow(out_img_eccv16)
	plt.title('Output (ECCV 16)')
	plt.axis('off')

	plt.subplot(2,2,4)
	plt.imshow(out_img_siggraph17)
	plt.title('Output (SIGGRAPH 17)')
	plt.axis('off')

	plt.show()

## DeOldify
We will now move on to the DeOldify models


In [4]:
# Set up divice
from deoldify import device
from deoldify.device_id import DeviceId
# choices:  CPU, GPU0...GPU7
# Again, I have only tested using a CPU
device.set(device=DeviceId.CPU)

<DeviceId.CPU: 99>

In [6]:
# Import functions
from deoldify.visualize import *
from deoldify.visualize import get_image_colorizer
plt.style.use('dark_background')
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

IsADirectoryError: [Errno 21] Is a directory: '/Users/hhayat002/Documents/UDA_test/UDA_Final_project_Hayat'

In [ ]:
#Adjust render_factor (int) if image doesn't look quite right (max 64 on 11GB GPU).  The default here works for most photos.
#It literally just is a number multiplied by 16 to get the square render resolution.
#Note that this doesn't affect the resolution of the final output- the output is the same resolution as the input.
#Example:  render_factor=21 => color is rendered at 16x21 = 336x336 px.

### Artistic model

In [25]:
# Default render factor is 35, however I achieved better results with 40 but takes more time to run
# Can potentially be increased further with a GPU

# Initialise artistic colouriser
vis_artistic = get_image_colorizer(render_factor=35, artistic=True)

/Users/hhayat002/Documents/MLDS/UDA/test2/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/Users/hhayat002/Documents/MLDS/UDA/test2/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Get synthetic black and white images
images = [join('test_images_bw/', f) for f in listdir('test_images_bw/') if basename(f).split('.')[1] in ['jpeg', 'jpg', 'png']]

In [ ]:
# Prepare output folders
try:
	mkdir('out_deoldify')
except FileExistsError:
	pass

try:
	mkdir('out_deoldify/Artistic')
except FileExistsError:
	pass

try:
	mkdir('out_deoldify/Stable')
except FileExistsError:
	pass

In [ ]:
# Apply Artistic model
# Default render factors for images themselves is 38. I achieved better results with 50.
for image in images:
	vis_artistic.plot_transformed_image(image, render_factor=38, results_dir=Path('out_deoldify/Artistic/'), watermarked=False)

# The results are included in the fourth column of Figure 6 in the pdf.

### Stable model

In [ ]:
# Default render factor is 35, but I achieved better results with 45

# Initialise stable colouriser
vis_stable = get_image_colorizer(render_factor=35, artistic=False)

In [ ]:
# Default render_factor:38
# Optimal solution achieved at 55
for image in images:
	vis_stable.plot_transformed_image(image, render_factor=38, results_dir=Path('out_deoldify/Stable/'), watermarked=False)


## Apply DeOldify to old photos

In [ ]:
images_old = [join('old_images/', f) for f in listdir('old_images/') if basename(f).split('.')[1] in ['jpeg', 'jpg', 'png']]

In [ ]:
# Figure 7 and 8 results from pdf:
for image in images_old:
	vis_artistic.plot_transformed_image(image, render_factor=38, results_dir=Path('out_deoldify/Artistic/'), watermarked=False)

In [ ]:
# Figure 7 and 8 results from pdf:
for image in images_old:
	vis_stable.plot_transformed_image(image, render_factor=38, results_dir=Path('out_deoldify/Stable/'), watermarked=False)

In [ ]:
b=datetime.now()
b-a